In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import folium
from folium import plugins
import time
import tensorflow as tf

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [3]:
uber_raw = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/uber-raw-data-janjune-15.csv', usecols=['Pickup_date', 'locationID'])
uber_raw['Pickup_date'] = pd.to_datetime(uber_raw['Pickup_date'])
# #Jan-Jun 2015

In [19]:
uber_raw.head()

,Pickup_date,Year,Month,Day,Hour,Min,locationID
0,2015-05-17 09:47:00,2015,5,17,9,47,141
1,2015-05-17 09:47:00,2015,5,17,9,47,65
2,2015-05-17 09:47:00,2015,5,17,9,47,100
3,2015-05-17 09:47:00,2015,5,17,9,47,80
4,2015-05-17 09:47:00,2015,5,17,9,47,90


In [4]:
uber_raw.insert(1, 'Min', uber_raw['Pickup_date'].map(lambda x: x.minute))
uber_raw.insert(1, 'Hour', uber_raw['Pickup_date'].map(lambda x: x.hour))
uber_raw.insert(1, 'Day', uber_raw['Pickup_date'].map(lambda x: x.day))
uber_raw.insert(1, 'Month', uber_raw['Pickup_date'].map(lambda x: x.month))
uber_raw.insert(1, 'Year', uber_raw['Pickup_date'].map(lambda x: x.year))

In [5]:
uber_raw[['Month','Day','Hour', 'Pickup_date']].sort_values(['Month', 'Day', 'Hour']) #Start_Date = 

,Month,Day,Hour,Pickup_date
44163,1,1,0,2015-01-01 00:41:02
44171,1,1,0,2015-01-01 00:01:37
44175,1,1,0,2015-01-01 00:50:41
44179,1,1,0,2015-01-01 00:15:57
44183,1,1,0,2015-01-01 00:51:18
44185,1,1,0,2015-01-01 00:25:30
44191,1,1,0,2015-01-01 00:22:46
44197,1,1,0,2015-01-01 00:57:20
44566,1,1,0,2015-01-01 00:49:52
44569,1,1,0,2015-01-01 00:42:13


In [6]:
#Jan 26:22-Jan 27:8 == Bad
#p_data = uber_raw[(uber_raw['Month'] == 1) & (uber_raw['Day'] == 26)]

In [7]:
#p_data.groupby(['Month', 'Day', 'Hour'])   #.sort_values(['Month', 'Day', 'Hour'])

In [14]:
#Jan 26:22-Jan 27:8 == Bad
# p_data = p_data.sort_values('Hour')
# P_data = p_data[p_data['Hour'] == 23]
# P_data

#RAW DATA DOES NOT CONTIAN THAT INFO

,Pickup_date,Year,Month,Day,Hour,Min,locationID


In [9]:
def tensor_loc(series):
    lst = []
    for cell in series:
        lst.append(cell)
    return lst

In [15]:
uber2 = uber_raw.groupby(['Year', 'Month','Day', 'Hour'])['locationID'].apply(tensor_loc).reset_index()

In [21]:
uber2.head()

,Year,Month,Day,Hour,locationID
0,2015,1,1,0,"[163, 164, 80, 144, 97, 230, 162, 87, 79, 159,..."
1,2015,1,1,1,"[142, 231, 113, 79, 158, 114, 113, 232, 112, 1..."
2,2015,1,1,2,"[164, 234, 148, 148, 145, 142, 48, 87, 148, 14..."
3,2015,1,1,3,"[41, 256, 148, 17, 79, 97, 238, 211, 113, 124,..."
4,2015,1,1,4,"[36, 158, 107, 36, 232, 113, 68, 164, 89, 143,..."


In [20]:
locationID_list = sorted(uber_raw['locationID'].unique())
###HA! The reason for the empty columns is because they NEVER appear in this unique list. So just add them as 0's

In [22]:
locationID_list

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187

In [ ]:
#del uber2['column_name']

In [30]:
#list(uber2['locationID'][0]).count(4) #58

58

In [36]:
#len(locationID_list) * 3.5 / 60

15.283333333333333

In [12]:
start_time = time.time()
for i in locationID_list: #Columns
    #print i
    for k in range(len(uber2)): #Rows
        uber2.ix[k, str(i)] = list(uber2['locationID'][k]).count(i)

print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'locationID_list' is not defined

In [39]:
#uber2.to_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/uber_loc_density.csv')

In [40]:
#uber2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4334 entries, 0 to 4333
Columns: 267 entries, Year to 265
dtypes: float64(262), int64(4), object(1)
memory usage: 8.8+ MB


In [ ]:
#

In [ ]:
#

In [ ]:
#

In [7]:
uber = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/uber_loc_density.csv')

In [ ]:
uber = uber.drop('Unnamed: 0', 1)
uber

In [ ]:
#623 = bad
#Jan 26:22-Jan 27:8 == Bad
counter = 0
total_count = 0
for i in uber['Hour']:
    #print i, counter
    
    if int(i) == counter:
        pass
    else:
        print 'bad', total_count

    if counter == 23:
        counter = 0
    else:
        counter += 1

    total_count += 1

In [134]:
uber

,Year,Month,Day,Hour,locationID,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
0,2015,1,1,0,"[163, 164, 80, 144, 97, 230, 162, 87, 79, 159,...",0.0,0.0,2.0,58.0,0.0,...,93.0,6.0,4.0,1.0,7.0,17.0,42.0,77.0,0.0,3.0
1,2015,1,1,1,"[142, 231, 113, 79, 158, 114, 113, 232, 112, 1...",0.0,0.0,1.0,39.0,0.0,...,83.0,16.0,7.0,3.0,3.0,13.0,31.0,50.0,0.0,6.0
2,2015,1,1,2,"[164, 234, 148, 148, 145, 142, 48, 87, 148, 14...",0.0,0.0,1.0,50.0,1.0,...,105.0,8.0,12.0,2.0,7.0,10.0,27.0,67.0,0.0,9.0
3,2015,1,1,3,"[41, 256, 148, 17, 79, 97, 238, 211, 113, 124,...",0.0,0.0,4.0,49.0,0.0,...,122.0,8.0,10.0,1.0,10.0,16.0,22.0,29.0,0.0,2.0
4,2015,1,1,4,"[36, 158, 107, 36, 232, 113, 68, 164, 89, 143,...",0.0,0.0,0.0,30.0,0.0,...,94.0,5.0,5.0,1.0,8.0,7.0,9.0,23.0,0.0,2.0
5,2015,1,1,5,"[89, 56, 231, 13, 142, 211, 37, 79, 132, 224, ...",0.0,0.0,1.0,22.0,0.0,...,77.0,0.0,4.0,2.0,7.0,6.0,6.0,6.0,0.0,1.0
6,2015,1,1,6,"[87, 49, 256, 68, 237, 17, 87, 33, 112, 79, 11...",0.0,0.0,0.0,4.0,0.0,...,63.0,4.0,1.0,2.0,4.0,2.0,6.0,11.0,0.0,0.0
7,2015,1,1,7,"[246, 112, 256, 226, 236, 239, 243, 246, 41, 1...",0.0,0.0,0.0,5.0,0.0,...,46.0,1.0,2.0,0.0,0.0,6.0,2.0,3.0,0.0,0.0
8,2015,1,1,8,"[138, 231, 61, 17, 14, 161, 75, 68, 48, 50, 61...",0.0,0.0,0.0,4.0,0.0,...,27.0,4.0,0.0,0.0,3.0,1.0,5.0,4.0,0.0,0.0
9,2015,1,1,9,"[225, 186, 256, 42, 50, 164, 80, 113, 200, 230...",0.0,0.0,0.0,3.0,0.0,...,15.0,0.0,3.0,0.0,3.0,2.0,4.0,7.0,0.0,0.0


In [1]:
for i in uber.columns:
    print i

NameError: name 'uber' is not defined

In [135]:
uber_feb_jun = uber[uber['Month'] > 1]

In [139]:
uber = uber_feb_jun.reset_index().drop('index', 1)

In [140]:
uber

,Year,Month,Day,Hour,locationID,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
0,2015,2,1,0,"[164, 107, 7, 239, 249, 50, 107, 137, 226, 79,...",0.0,0.0,4.0,39.0,0.0,...,82.0,10.0,1.0,2.0,8.0,18.0,20.0,59.0,0.0,0.0
1,2015,2,1,1,"[162, 211, 79, 107, 198, 188, 162, 255, 107, 2...",0.0,0.0,0.0,58.0,0.0,...,95.0,5.0,4.0,0.0,7.0,8.0,10.0,27.0,0.0,0.0
2,2015,2,1,2,"[79, 148, 158, 158, 226, 82, 249, 36, 48, 198,...",0.0,0.0,0.0,30.0,0.0,...,61.0,2.0,0.0,3.0,6.0,8.0,6.0,20.0,0.0,0.0
3,2015,2,1,3,"[255, 106, 114, 80, 216, 49, 17, 97, 48, 164, ...",0.0,0.0,0.0,23.0,0.0,...,40.0,0.0,2.0,0.0,3.0,3.0,8.0,14.0,0.0,1.0
4,2015,2,1,4,"[61, 80, 22, 69, 50, 87, 246, 144, 48, 233, 21...",0.0,0.0,0.0,11.0,0.0,...,16.0,1.0,3.0,0.0,1.0,5.0,2.0,2.0,0.0,0.0
5,2015,2,1,5,"[186, 37, 249, 132, 36, 50, 112, 255, 37, 255,...",0.0,0.0,0.0,9.0,0.0,...,14.0,1.0,1.0,0.0,1.0,2.0,6.0,4.0,0.0,0.0
6,2015,2,1,6,"[37, 215, 239, 141, 132, 107, 17, 4, 230, 107,...",0.0,0.0,0.0,9.0,0.0,...,12.0,2.0,1.0,0.0,3.0,1.0,5.0,8.0,0.0,0.0
7,2015,2,1,7,"[79, 224, 239, 260, 123, 129, 13, 256, 36, 79,...",0.0,0.0,1.0,10.0,0.0,...,12.0,3.0,2.0,0.0,1.0,1.0,6.0,9.0,0.0,0.0
8,2015,2,1,8,"[223, 15, 127, 137, 107, 37, 48, 263, 4, 263, ...",0.0,0.0,0.0,8.0,0.0,...,25.0,1.0,2.0,0.0,2.0,4.0,12.0,19.0,0.0,0.0
9,2015,2,1,9,"[151, 255, 121, 51, 246, 238, 161, 142, 141, 2...",0.0,0.0,0.0,17.0,0.0,...,21.0,2.0,1.0,1.0,5.0,3.0,21.0,28.0,0.0,2.0


In [111]:
#Year/Month/Day
lst = []
for i in range(1,31): # Num of days plus 1
    weather = pd.read_csv('https://www.wunderground.com/history/airport/KNYC/2015/6/'+str(i)+'/DailyHistory.html?req_city=New+York&req_state=NY&req_statename=New+York&reqdb.zip=10001&reqdb.magic=10&reqdb.wmo=99999&format=1')
    lst.append(weather)

jun2015_weather = pd.concat(lst)

In [113]:
#jun2015_weather.to_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/jun2015_weather.csv')

In [9]:
jan2015_weather = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/jan2015_weather.csv')
feb2015_weather = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/feb2015_weather.csv')
mar2015_weather = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/mar2015_weather.csv')
apr2015_weather = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/apr2015_weather.csv')
may2015_weather = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/may2015_weather.csv')
jun2015_weather = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/jun2015_weather.csv')

In [10]:
total_weather = pd.concat([jan2015_weather, feb2015_weather, mar2015_weather,
                         apr2015_weather, may2015_weather, jun2015_weather])

In [127]:
new_weather = pd.concat([feb2015_weather, mar2015_weather,
                         apr2015_weather, may2015_weather, jun2015_weather])

In [128]:
def time_conv(x):
    x = x[11:13]
    
    if x == '01':
        return '20'
    elif x == '02':
        return '21'
    elif x == '03':
        return '22'
    elif x == '04':
        return '23'
    elif x == '05':
        return '00'
    elif x == '06':
        return '01'
    elif x == '07':
        return '02'
    elif x == '08':
        return '03'
    elif x == '09':
        return '04'
    elif x == '10':
        return '05'
    elif x == '11':
        return '06'
    elif x == '12':
        return '07'
    elif x == '13':
        return '08'
    elif x == '14':
        return '09'
    elif x == '15':
        return '10'
    elif x == '16':
        return '11'
    elif x == '17':
        return '12'
    elif x == '18':
        return '13'
    elif x == '19':
        return '14'
    elif x == '20':
        return '15'
    elif x == '21':
        return '16'
    elif x == '22':
        return '17'
    elif x == '23':
        return '18'
    elif x == '00':
        return '19'
    else:
        return 'ERROR'

In [129]:
#total_weather = total_weather.drop('Time', 1)
new_weather.insert(0, 'Time', new_weather['DateUTC<br />'].map(time_conv))

In [130]:
Time = list(new_weather['Time'])
Time2 = []
total_count = 0
counter = 0
prev = 0
error2_counter = 0
for i in Time:
    #print int(i), counter
    
    if prev == i:
        #print 'caught_1'
        Time2.append(np.nan)
        continue
    else:
        prev = i
    
    if int(i) == counter:
        Time2.append(i)
    else:
        Time2.append('ERROR2')
        counter += +1
        print '=============================='
        error2_counter += 1
        pass
    
    if counter == 23:
        counter = 0
    else:
        counter += 1
    
    total_count += 1

print error2_counter

1


In [131]:
new_weather.insert(0, 'Time_Unique', Time2)
new_weather = new_weather.reset_index().drop('index', 1)
new_weather = new_weather[new_weather['Time_Unique'].notnull()]
new_weather = new_weather.reset_index().drop('index', 1)

In [132]:
new_weather #Feb-Jun Data (5 MONTHS)

,Time_Unique,Time,Conditions,DateUTC<br />,Dew PointF,Events,Gust SpeedMPH,Humidity,PrecipitationIn,Sea Level PressureIn,TemperatureF,TimeEDT,TimeEST,Unnamed: 0,VisibilityMPH,Wind Direction,Wind SpeedMPH,WindDirDegrees
0,00,00,Clear,2015-02-01 05:51:00<br />,6.1,NaN,-,55.0,NaN,30.19,19.9,NaN,12:51 AM,0,10.0,Calm,Calm,0
1,01,01,Partly Cloudy,2015-02-01 06:51:00<br />,6.1,NaN,-,53.0,NaN,30.19,21.0,NaN,1:51 AM,1,10.0,WSW,3.5,250
2,02,02,Clear,2015-02-01 07:51:00<br />,6.1,NaN,-,51.0,NaN,30.19,21.9,NaN,2:51 AM,2,10.0,Variable,3.5,0
3,03,03,Mostly Cloudy,2015-02-01 08:51:00<br />,7.0,NaN,-,50.0,NaN,30.19,23.0,NaN,3:51 AM,3,10.0,Variable,3.5,0
4,04,04,Scattered Clouds,2015-02-01 09:51:00<br />,8.1,NaN,-,51.0,NaN,30.19,24.1,NaN,4:51 AM,4,10.0,North,-9999.0,0
5,05,05,Overcast,2015-02-01 10:51:00<br />,8.1,NaN,-,51.0,NaN,30.21,24.1,NaN,5:51 AM,5,10.0,WSW,8.1,250
6,06,06,Overcast,2015-02-01 11:51:00<br />,10.0,NaN,-,55.0,NaN,30.21,24.1,NaN,6:51 AM,6,10.0,Calm,Calm,0
7,07,07,Overcast,2015-02-01 12:51:00<br />,12.0,NaN,-,55.0,NaN,30.22,26.1,NaN,7:51 AM,7,10.0,Variable,3.5,0
8,08,08,Overcast,2015-02-01 13:51:00<br />,14.0,NaN,-,56.0,NaN,30.22,28.0,NaN,8:51 AM,8,10.0,Calm,Calm,0
9,09,09,Clear,2015-02-01 14:51:00<br />,15.1,NaN,-,52.0,NaN,30.21,30.9,NaN,9:51 AM,9,9.0,Variable,4.6,0


In [144]:
error = new_weather[new_weather['Time_Unique'] == 'ERROR2'] #LINE 635

In [155]:
final_weather = pd.concat([new_weather[:635], new_weather[634:635], new_weather[634:635], new_weather[636:]])

In [169]:
final_weather = final_weather.drop('index',1 )

In [181]:
final_weather.ix[635, 'Time'] = 11

In [191]:
weather = weather.drop(['Time_Unique', 'Time', 'DateUTC<br />', 'TimeEDT', 'TimeEST', 'Unnamed: 0'], 1)

In [204]:
index = []
for i in range(0,len(weather)):
    index.append(i)

In [205]:
print len(index)
print len(uber)

3599
3599


In [206]:
uber['Index'] = index
uber

,Year,Month,Day,Hour,locationID,1,2,3,4,5,...,257,258,259,260,261,262,263,264,265,Index
0,2015,2,1,0,"[164, 107, 7, 239, 249, 50, 107, 137, 226, 79,...",0.0,0.0,4.0,39.0,0.0,...,10.0,1.0,2.0,8.0,18.0,20.0,59.0,0.0,0.0,0
1,2015,2,1,1,"[162, 211, 79, 107, 198, 188, 162, 255, 107, 2...",0.0,0.0,0.0,58.0,0.0,...,5.0,4.0,0.0,7.0,8.0,10.0,27.0,0.0,0.0,1
2,2015,2,1,2,"[79, 148, 158, 158, 226, 82, 249, 36, 48, 198,...",0.0,0.0,0.0,30.0,0.0,...,2.0,0.0,3.0,6.0,8.0,6.0,20.0,0.0,0.0,2
3,2015,2,1,3,"[255, 106, 114, 80, 216, 49, 17, 97, 48, 164, ...",0.0,0.0,0.0,23.0,0.0,...,0.0,2.0,0.0,3.0,3.0,8.0,14.0,0.0,1.0,3
4,2015,2,1,4,"[61, 80, 22, 69, 50, 87, 246, 144, 48, 233, 21...",0.0,0.0,0.0,11.0,0.0,...,1.0,3.0,0.0,1.0,5.0,2.0,2.0,0.0,0.0,4
5,2015,2,1,5,"[186, 37, 249, 132, 36, 50, 112, 255, 37, 255,...",0.0,0.0,0.0,9.0,0.0,...,1.0,1.0,0.0,1.0,2.0,6.0,4.0,0.0,0.0,5
6,2015,2,1,6,"[37, 215, 239, 141, 132, 107, 17, 4, 230, 107,...",0.0,0.0,0.0,9.0,0.0,...,2.0,1.0,0.0,3.0,1.0,5.0,8.0,0.0,0.0,6
7,2015,2,1,7,"[79, 224, 239, 260, 123, 129, 13, 256, 36, 79,...",0.0,0.0,1.0,10.0,0.0,...,3.0,2.0,0.0,1.0,1.0,6.0,9.0,0.0,0.0,7
8,2015,2,1,8,"[223, 15, 127, 137, 107, 37, 48, 263, 4, 263, ...",0.0,0.0,0.0,8.0,0.0,...,1.0,2.0,0.0,2.0,4.0,12.0,19.0,0.0,0.0,8
9,2015,2,1,9,"[151, 255, 121, 51, 246, 238, 161, 142, 141, 2...",0.0,0.0,0.0,17.0,0.0,...,2.0,1.0,1.0,5.0,3.0,21.0,28.0,0.0,2.0,9


In [207]:
weather['Index'] = index
weather

,Conditions,Dew PointF,Events,Gust SpeedMPH,Humidity,PrecipitationIn,Sea Level PressureIn,TemperatureF,VisibilityMPH,Wind Direction,Wind SpeedMPH,WindDirDegrees,Index
0,Clear,6.1,NaN,-,55.0,NaN,30.19,19.9,10.0,Calm,Calm,0,0
1,Partly Cloudy,6.1,NaN,-,53.0,NaN,30.19,21.0,10.0,WSW,3.5,250,1
2,Clear,6.1,NaN,-,51.0,NaN,30.19,21.9,10.0,Variable,3.5,0,2
3,Mostly Cloudy,7.0,NaN,-,50.0,NaN,30.19,23.0,10.0,Variable,3.5,0,3
4,Scattered Clouds,8.1,NaN,-,51.0,NaN,30.19,24.1,10.0,North,-9999.0,0,4
5,Overcast,8.1,NaN,-,51.0,NaN,30.21,24.1,10.0,WSW,8.1,250,5
6,Overcast,10.0,NaN,-,55.0,NaN,30.21,24.1,10.0,Calm,Calm,0,6
7,Overcast,12.0,NaN,-,55.0,NaN,30.22,26.1,10.0,Variable,3.5,0,7
8,Overcast,14.0,NaN,-,56.0,NaN,30.22,28.0,10.0,Calm,Calm,0,8
9,Clear,15.1,NaN,-,52.0,NaN,30.21,30.9,9.0,Variable,4.6,0,9


In [208]:
uber_data = uber.merge(weather)
uber_data

,Year,Month,Day,Hour,locationID,1,2,3,4,5,...,Events,Gust SpeedMPH,Humidity,PrecipitationIn,Sea Level PressureIn,TemperatureF,VisibilityMPH,Wind Direction,Wind SpeedMPH,WindDirDegrees
0,2015,2,1,0,"[164, 107, 7, 239, 249, 50, 107, 137, 226, 79,...",0.0,0.0,4.0,39.0,0.0,...,NaN,-,55.0,NaN,30.19,19.9,10.0,Calm,Calm,0
1,2015,2,1,1,"[162, 211, 79, 107, 198, 188, 162, 255, 107, 2...",0.0,0.0,0.0,58.0,0.0,...,NaN,-,53.0,NaN,30.19,21.0,10.0,WSW,3.5,250
2,2015,2,1,2,"[79, 148, 158, 158, 226, 82, 249, 36, 48, 198,...",0.0,0.0,0.0,30.0,0.0,...,NaN,-,51.0,NaN,30.19,21.9,10.0,Variable,3.5,0
3,2015,2,1,3,"[255, 106, 114, 80, 216, 49, 17, 97, 48, 164, ...",0.0,0.0,0.0,23.0,0.0,...,NaN,-,50.0,NaN,30.19,23.0,10.0,Variable,3.5,0
4,2015,2,1,4,"[61, 80, 22, 69, 50, 87, 246, 144, 48, 233, 21...",0.0,0.0,0.0,11.0,0.0,...,NaN,-,51.0,NaN,30.19,24.1,10.0,North,-9999.0,0
5,2015,2,1,5,"[186, 37, 249, 132, 36, 50, 112, 255, 37, 255,...",0.0,0.0,0.0,9.0,0.0,...,NaN,-,51.0,NaN,30.21,24.1,10.0,WSW,8.1,250
6,2015,2,1,6,"[37, 215, 239, 141, 132, 107, 17, 4, 230, 107,...",0.0,0.0,0.0,9.0,0.0,...,NaN,-,55.0,NaN,30.21,24.1,10.0,Calm,Calm,0
7,2015,2,1,7,"[79, 224, 239, 260, 123, 129, 13, 256, 36, 79,...",0.0,0.0,1.0,10.0,0.0,...,NaN,-,55.0,NaN,30.22,26.1,10.0,Variable,3.5,0
8,2015,2,1,8,"[223, 15, 127, 137, 107, 37, 48, 263, 4, 263, ...",0.0,0.0,0.0,8.0,0.0,...,NaN,-,56.0,NaN,30.22,28.0,10.0,Calm,Calm,0
9,2015,2,1,9,"[151, 255, 121, 51, 246, 238, 161, 142, 141, 2...",0.0,0.0,0.0,17.0,0.0,...,NaN,-,52.0,NaN,30.21,30.9,9.0,Variable,4.6,0


In [215]:
uber_data.to_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/uber_data.csv')

In [216]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3599 entries, 0 to 3598
Columns: 279 entries, Year to WindDirDegrees
dtypes: float64(268), int64(5), object(6)
memory usage: 7.7+ MB


In [218]:
uber_data.shape

(3599, 279)

In [2]:
uber = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/uber_data.csv')

In [8]:
#103
#104
#199

for i in uber.columns:
    print i

Year
Month
Day
Hour
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265


In [4]:
#uber['Year'].shape
zeros = np.zeros(shape=(3599,))

In [5]:
uber = uber.drop(['Unnamed: 0', 'locationID', 'Conditions', 'Dew PointF',
          'Events', 'Gust SpeedMPH','Humidity', 'PrecipitationIn',
                 'Sea Level PressureIn', 'TemperatureF','VisibilityMPH',
                 'Wind Direction', 'Wind SpeedMPH', 'WindDirDegrees'], 1)

In [6]:
#uber

In [7]:
#1 = 4, 2 = 5
uber.insert(106, '103', zeros) #Worked
uber.insert(107, '104', zeros) #Worked
uber.insert(202, '199', zeros)

In [9]:
uber.to_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/uber-pickups-in-new-york-city/loc_only_uber_agg.csv')